In [1]:
from bertopic import BERTopic
import statistics
import numpy as np
import json

# First we import the files
our_flyers_path = 'flyers_text.json'

with open(our_flyers_path, 'r') as file:
    json_file = json.load(file)

# A section
#a = json_file["(Langdurig) gebroken vliezen"]['Check uw dossier op MijnZGT']

# The list of sections to analyze is the list of values of each value of the original dictionary (json file)
sections_dict = {} # The dictionary is to avoid repeated elements
saw_sections = []
counter = -1
for document in json_file.values(): 
    counter += 1
    for section in document.values(): 
        if section not in saw_sections:
            sections_dict[section] = counter
            saw_sections.append(section)

'''
SOME OF THE SECTIONS ARE REPEATED, SO THERE'S LITTLE POINT ON KEEPING THEM
'''
sections = list(sections_dict.keys())
document_id_list = list(sections_dict.values())

print("Quantity of sections:", len(sections))

2024-01-06 13:39:13.879993: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 13:39:13.979683: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-06 13:39:14.581822: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-01-06 13:39:14.581908

Quantity of sections: 11827


In [ ]:
from bertopic.backend import BaseEmbedder
from sentence_transformers import SentenceTransformer
from umap import UMAP

class CustomEmbedder(BaseEmbedder):
    def __init__(self, embedding_model):
        super().__init__()
        self.embedding_model = embedding_model

    def embed(self, documents, verbose=False):
        embeddings = self.embedding_model.encode(documents, show_progress_bar=verbose)
        return embeddings 
    
def build_models(custom_embedder, top_n_words, min_topic_size, dimensions, def_top, def_size, def_dim): # The three inputs will be lists
    
    counter = 1
    for parameter in top_n_words:

        umap_model = UMAP(n_neighbors=15, n_components=def_dim, min_dist=0.0, metric='cosine')

        topic_model = BERTopic(embedding_model=custom_embedder, umap_model=umap_model, top_n_words=parameter, min_topic_size=def_size, language="multilingual", calculate_probabilities=True, verbose=True)
        topics, probs = topic_model.fit_transform(sections)

        topic_model.save("models/top_n_words_model_"+str(counter))
        
        counter += 1
        
    counter = 1
    for parameter in min_topic_size:

        umap_model = UMAP(n_neighbors=15, n_components=def_dim, min_dist=0.0, metric='cosine')

        topic_model = BERTopic(embedding_model=custom_embedder, umap_model=umap_model, top_n_words=def_top, min_topic_size=parameter, language="multilingual", calculate_probabilities=True, verbose=True)
        topics, probs = topic_model.fit_transform(sections)

        topic_model.save("models/min_topic_size_model_"+str(counter))
        
        counter += 1
        
    counter = 1
    for parameter in dimensions:

        umap_model = UMAP(n_neighbors=15, n_components=parameter, min_dist=0.0, metric='cosine')

        topic_model = BERTopic(embedding_model=custom_embedder, umap_model=umap_model, top_n_words=def_top, min_topic_size=def_size, language="multilingual", calculate_probabilities=True, verbose=True)
        topics, probs = topic_model.fit_transform(sections)

        topic_model.save("models/reduction_dim_model_"+str(counter))
        
        counter += 1
    
# Create custom backend
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
custom_embedder = CustomEmbedder(embedding_model=embedding_model)

top_n_words = [5, 10, 15, 20, 25, 30]
def_top = 10
min_topic_size = [10, 15, 20, 25, 30, 35, 40, 45, 50]
def_size = 10
dimensions = [5, 10, 20, 30, 40, 50, 60, 70, 80]
def_dim = 5
build_models(custom_embedder, top_n_words, min_topic_size, dimensions, def_top, def_size, def_dim)

2024-01-06 13:54:15,144 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/370 [00:00<?, ?it/s]

2024-01-06 13:55:34,756 - BERTopic - Embedding - Completed ✓
2024-01-06 13:55:34,757 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-06 13:56:05,173 - BERTopic - Dimensionality - Completed ✓
2024-01-06 13:56:05,177 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-06 13:56:21,010 - BERTopic - Cluster - Completed ✓
2024-01-06 13:56:21,020 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-06 13:56:22,108 - BERTopic - Representation - Completed ✓
2024-01-06 13:56:22,812 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
2024-01-06 13:56:35,525 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/370 [00:00<?, ?it/s]

2024-01-06 13:57:45,222 - BERTopic - Embedding - Completed ✓
2024-01-06 13:57:45,224 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-06 13:57:58,611 - BERTopic - Dimensionality - Completed ✓
2024-01-06 13:57:58,614 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-06 13:58:15,862 - BERTopic - Cluster - Completed ✓
2024-01-06 13:58:15,870 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-06 13:58:16,976 - BERTopic - Representation - Completed ✓
2024-01-06 13:58:17,685 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
2024-01-06 13:58:20,234 - BERTopic - Embedding - Transforming documents to embeddings.


...

Batches:   0%|          | 0/370 [00:00<?, ?it/s]

2024-01-06 14:27:41,999 - BERTopic - Embedding - Completed ✓
2024-01-06 14:27:42,001 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-06 14:27:54,320 - BERTopic - Dimensionality - Completed ✓
2024-01-06 14:27:54,323 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-06 14:28:11,763 - BERTopic - Cluster - Completed ✓
2024-01-06 14:28:11,771 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-06 14:28:12,844 - BERTopic - Representation - Completed ✓
2024-01-06 14:28:13,500 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
2024-01-06 14:28:16,269 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/370 [00:00<?, ?it/s]

2024-01-06 14:29:30,306 - BERTopic - Embedding - Completed ✓
2024-01-06 14:29:30,308 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-06 14:29:42,422 - BERTopic - Dimensionality - Completed ✓
2024-01-06 14:29:42,426 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-06 14:29:59,664 - BERTopic - Cluster - Completed ✓
2024-01-06 14:29:59,671 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-06 14:30:00,766 - BERTopic - Representation - Completed ✓
2024-01-06 14:30:01,409 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
2024-01-06 14:30:03,780 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/370 [00:00<?, ?it/s]